In [ ]:
PYTORCH_NO_CUDA_MEMORY_CACHING=1

from pathlib import Path
import matplotlib.pyplot as plt
import cv2
import torch
from torch import cuda
import os
import numpy as np
import random
from PIL import Image
import matplotlib.colors as mcolors
import numpy.ma as ma
import json
np.set_printoptions(precision=15)

# Ensure deterministic behavior (cannot control everything though)
torch.backends.cudnn.deterministic = True
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# !nvidia-smi

In [ ]:
dir_ = '/workspace/raid/OM_DeepLearning/MobileSAM-master/MobileSAMv2/'

In [ ]:
import argparse
import ast
import torch
from PIL import Image
import cv2
import os
import sys

sys.path.append('/workspace/raid/OM_DeepLearning/MobileSAM-master/MobileSAMv2/')

from mobilesamv2.promt_mobilesamv2 import ObjectAwareModel
from mobilesamv2 import sam_model_registry, SamPredictor
from typing import Any, Dict, Generator,List
import matplotlib.pyplot as plt
import numpy as np

def create_model():
    Prompt_guided_path=dir_+'PromptGuidedDecoder/Prompt_guided_Mask_Decoder.pt'
    obj_model_path=dir_+'weight/ObjectAwareModel.pt'
    ObjAwareModel = ObjectAwareModel(obj_model_path)
    PromptGuidedDecoder=sam_model_registry['PromptGuidedDecoder'](Prompt_guided_path)
    mobilesamv2 = sam_model_registry['vit_h']()
    mobilesamv2.prompt_encoder=PromptGuidedDecoder['PromtEncoder']
    mobilesamv2.mask_decoder=PromptGuidedDecoder['MaskDecoder']
    return mobilesamv2,ObjAwareModel
    
def show_anns(anns):
    if len(anns) == 0:
        return
    ax = plt.gca()
    ax.set_autoscale_on(False)
    img = np.ones((anns.shape[1], anns.shape[2], 4))
    img[:,:,3] = 0
    for ann in range(anns.shape[0]):
        m = anns[ann].bool()
        m=m.cpu().numpy()
        color_mask = np.concatenate([np.random.random(3), [1]])
        img[m] = color_mask
    ax.imshow(img)

def batch_iterator(batch_size: int, *args) -> Generator[List[Any], None, None]:
    assert len(args) > 0 and all(
        len(a) == len(args[0]) for a in args
    ), "Batched iteration must have inputs of all the same size."
    n_batches = len(args[0]) // batch_size + int(len(args[0]) % batch_size != 0)
    for b in range(n_batches):
        yield [arg[b * batch_size : (b + 1) * batch_size] for arg in args]

encoder_path={'efficientvit_l2':dir_+'weight/l2.pt',
            'tiny_vit':dir_+'weight/mobile_sam.pt',
            'sam_vit_h':dir_+'weight/sam_vit_h.pt',}

In [ ]:
# # Load bboxes from source detection & deblending
# import json
# import time

# # image_path = '/workspace/raid/OM_DeepLearning/XMM_OM_code/scaled_raw/clahe_S0720251301_L.png'

# def remove_boxes_near_corners(boxes, img_width, img_height, distance):
#     """
#     Remove bounding boxes that are within a certain distance from the corners of the image.

#     Args:
#         boxes (list): List of bounding boxes. Each box is a tuple (x1, y1, x2, y2).
#         img_width (int): Width of the image.
#         img_height (int): Height of the image.
#         distance (int): The distance from the corners.

#     Returns:
#         list: The list of remaining bounding boxes.
#     """
    
#     corners = [(0, 0), (img_width, 0), (0, img_height), (img_width, img_height)]
#     new_boxes = []
#     for box in boxes:
#         x1, y1, x2, y2 = box
#         box_center = ((x1 + x2) / 2, (y1 + y2) / 2)
#         if all(np.sqrt((cx - box_center[0]) ** 2 + (cy - box_center[1]) ** 2) > distance for cx, cy in corners):
#             new_boxes.append(box)
#     return new_boxes
    
# with open('extracted_sources_bboxes_points.json', 'r') as f:
#     extracted_bboxes_points = json.load(f)

In [ ]:
# extracted_bboxes_points['bboxes']

In [ ]:

# import torch
# import random

# def generate_random_data(batch_size, num_points, image_size=(256, 256), num_boxes=2):
#     batched_input = []

#     for _ in range(batch_size):
#         image = torch.randn((3, image_size[0], image_size[1]))  # 3xHxW format
#         original_size = (random.randint(100, 500), random.randint(100, 500))
#         point_coords = torch.randn((batch_size, num_points, 2))  # BxNx2
#         point_labels = torch.randint(0, 2, (batch_size, num_points))  # BxN
#         boxes = torch.randn((num_boxes, 4))  # Bx4
#         mask_inputs = torch.randn((1, 1, image_size[0], image_size[1]))  # Bx1xHxW

#         input_dict = {
#             'image': image,
#             'original_size': original_size,
#             'point_coords': point_coords,
#             'point_labels': point_labels,
#             'boxes': boxes,
#             'mask_inputs': mask_inputs,
#         }

#         batched_input.append(input_dict)

#     return batched_input

# batch_size = 2
# num_points = 5
# random_data = generate_random_data(batch_size, num_points)

# import numpy as np
# batched_input = np.array([random_data, random_data])

# from torchview import draw_graph

# model_graph = draw_graph(mobilesamv2, input_data = random_data, depth= 3, \
#                          device = 'cuda:7', save_graph=True, filename='./view_sam', \
#                         hide_module_functions=False, hide_inner_tensors=True, roll=True)
# # model_graph.visual_graph

In [ ]:
class Args:
    ObjectAwareModel_path = dir_+'PromptGuidedDecoder/ObjectAwareModel.pt'
    Prompt_guided_Mask_Decoder_path = dir_+'PromptGuidedDecoder/Prompt_guided_Mask_Decoder.pt'
    encoder_path = dir_
    img_path = '/workspace/raid/OM_DeepLearning/XMM_OM_dataset/zscaled_512_rescaled_SAM_stats/'
    imgsz = 512
    iou = 0.9
    conf = 0.4
    retina = False
    output_dir = '/workspace/raid/OM_DeepLearning/new_temp/'
    encoder_type = 'tiny_vit'  # choose from ['tiny_vit','sam_vit_h','mobile_sam','efficientvit_l2','efficientvit_l1','efficientvit_l0']

args = Args()

# import pdb;pdb.set_trace()
output_dir=args.output_dir  
mobilesamv2, ObjAwareModel=create_model()
image_encoder=sam_model_registry[args.encoder_type](encoder_path[args.encoder_type])
mobilesamv2.image_encoder=image_encoder
device = "cuda:7" if torch.cuda.is_available() else "cpu"
mobilesamv2.to(device=device)
mobilesamv2.eval()
predictor = SamPredictor(mobilesamv2)
image_files= os.listdir(args.img_path)

In [ ]:
import argparse
import ast
import torch
from PIL import Image
import cv2
import os
import sys
from mobilesamv2.promt_mobilesamv2 import ObjectAwareModel
from mobilesamv2 import sam_model_registry, SamPredictor
from typing import Any, Dict, Generator,List
import matplotlib.pyplot as plt
import numpy as np
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--ObjectAwareModel_path", type=str, default='./PromptGuidedDecoder/ObjectAwareModel.pt', help="ObjectAwareModel path")
    parser.add_argument("--Prompt_guided_Mask_Decoder_path", type=str, default='./PromptGuidedDecoder/Prompt_guided_Mask_Decoder.pt', help="Prompt_guided_Mask_Decoder path")
    parser.add_argument("--encoder_path", type=str, default="./", help="select your own path")
    parser.add_argument("--img_path", type=str, default="./test_images/", help="path to image file")
    parser.add_argument("--imgsz", type=int, default=1024, help="image size")
    parser.add_argument("--iou",type=float,default=0.9,help="yolo iou")
    parser.add_argument("--conf", type=float, default=0.4, help="yolo object confidence threshold")
    parser.add_argument("--retina",type=bool,default=True,help="draw segmentation masks",)
    parser.add_argument("--output_dir", type=str, default="./", help="image save path")
    parser.add_argument("--encoder_type", choices=['tiny_vit','sam_vit_h','mobile_sam','efficientvit_l2','efficientvit_l1','efficientvit_l0'], help="choose the model type")
    return parser.parse_args()

def create_model():
    Prompt_guided_path=dir_+'PromptGuidedDecoder/Prompt_guided_Mask_Decoder.pt'
    obj_model_path=dir_+'weight/ObjectAwareModel.pt'
    ObjAwareModel = ObjectAwareModel(obj_model_path)
    PromptGuidedDecoder=sam_model_registry['PromptGuidedDecoder'](Prompt_guided_path)
    mobilesamv2 = sam_model_registry['vit_h']()
    mobilesamv2.prompt_encoder=PromptGuidedDecoder['PromtEncoder']
    mobilesamv2.mask_decoder=PromptGuidedDecoder['MaskDecoder']
    return mobilesamv2,ObjAwareModel
    
def show_anns(anns):
    if len(anns) == 0:
        return
    ax = plt.gca()
    # ax.set_autoscale_on(False)
    img = np.ones((anns.shape[1], anns.shape[2], 4))
    img[:,:,3] = 0
    for ann in range(anns.shape[0]):
        m = anns[ann].bool()
        m=m.cpu().numpy()
        color_mask = np.concatenate([np.random.random(3), [1]])
        img[m] = color_mask
    ax.imshow(img)

def batch_iterator(batch_size: int, *args) -> Generator[List[Any], None, None]:
    assert len(args) > 0 and all(
        len(a) == len(args[0]) for a in args
    ), "Batched iteration must have inputs of all the same size."
    n_batches = len(args[0]) // batch_size + int(len(args[0]) % batch_size != 0)
    for b in range(n_batches):
        yield [arg[b * batch_size : (b + 1) * batch_size] for arg in args]


encoder_path={'efficientvit_l2':dir_+'weight/l2.pt',
            'tiny_vit':dir_+'weight/mobile_sam.pt',
            'sam_vit_h':dir_+'weight/sam_vit_h.pt',}

def main(args):
    # import pdb;pdb.set_trace()
    output_dir=args.output_dir  
    mobilesamv2, ObjAwareModel=create_model()
    image_encoder=sam_model_registry[args.encoder_type](encoder_path[args.encoder_type])
    mobilesamv2.image_encoder=image_encoder
    device = "cuda" if torch.cuda.is_available() else "cpu"
    mobilesamv2.to(device=device)
    mobilesamv2.eval()
    predictor = SamPredictor(mobilesamv2)
    image_files= os.listdir(args.img_path)
    for image_name in image_files[:10]:
        print('*****', image_name)
        image = cv2.imread(args.img_path + image_name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        obj_results = ObjAwareModel(image,device=device,retina_masks=args.retina,imgsz=args.imgsz,conf=args.conf,iou=args.iou)
        predictor.set_image(image)
        input_boxes1 = obj_results[0].boxes.xyxy
        input_boxes = input_boxes1.cpu().numpy()
        input_boxes = predictor.transform.apply_boxes(input_boxes, predictor.original_size)
        input_boxes = torch.from_numpy(input_boxes).to(device)
        sam_mask=[]
        image_embedding=predictor.features
        image_embedding=torch.repeat_interleave(image_embedding, 720, dim=0)
        prompt_embedding=mobilesamv2.prompt_encoder.get_dense_pe()
        prompt_embedding=torch.repeat_interleave(prompt_embedding, 720, dim=0)
        for (boxes,) in batch_iterator(320, input_boxes):
            with torch.no_grad():
                # Create a black image of size 1024x1024
                image1 = np.zeros((1024, 1024, 3), dtype=np.uint8)
                
                for bbox in boxes:
                    # Each bbox is a tuple or list in the format (x1, y1, x2, y2)
                    x1, y1, x2, y2 = bbox.detach().cpu().numpy()
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                
                    # Draw rectangle (bounding box) on the image
                    cv2.rectangle(image1, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green rectangle with thickness 2
                
                # Convert the BGR image to RGB
                image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
                
                # Display the image
                plt.imshow(image1)
                plt.show()
                plt.close()

                image_embedding=image_embedding[0:boxes.shape[0],:,:,:]
                prompt_embedding=prompt_embedding[0:boxes.shape[0],:,:,:]
                sparse_embeddings, dense_embeddings = mobilesamv2.prompt_encoder(
                    points=None,
                    boxes=boxes,
                    masks=None,)
                low_res_masks, _ = mobilesamv2.mask_decoder(
                    image_embeddings=image_embedding,
                    image_pe=prompt_embedding,
                    sparse_prompt_embeddings=sparse_embeddings,
                    dense_prompt_embeddings=dense_embeddings,
                    multimask_output=False,
                    simple_type=True,
                )
                low_res_masks=predictor.model.postprocess_masks(low_res_masks, predictor.input_size, predictor.original_size)
                sam_mask_pre = (low_res_masks > mobilesamv2.mask_threshold)*1.0
                sam_mask.append(sam_mask_pre.squeeze(1))
                # print(sam_mask[0].shape)
        sam_mask=torch.cat(sam_mask)
        # print(sam_mask.shape)
        annotation = sam_mask
        areas = torch.sum(annotation, dim=(1, 2))
        sorted_indices = torch.argsort(areas, descending=True)
        show_img = annotation[sorted_indices]
        plt.figure(figsize=(5,5))
        background=np.ones_like(image)*255
        plt.imshow(background)
        # print('torch', show_img.shape)
        show_anns(show_img)
        plt.axis('off')
        plt.show() 
        plt.savefig("{}".format(output_dir+image_name), bbox_inches='tight', pad_inches = 0.0) 
        plt.close()

args = Args()
main(args)

In [ ]:
plt.imshow(obj_results[0].masks.data[0].permute(1,2,0).detach().cpu().numpy()*255)
plt.show()

In [ ]:
obj_results[0].boxes.data

In [ ]:
# output = sam_model(sample_batched_input, multimask_output=True)

# # Visualize the computation graph using torchview
# draw_graph(sam_model, input_size=(1, 3, 256, 256))

In [ ]:
# cd MobileSAMv2
bash ./experiments/mobilesamv2.sh